# MTL 計算效益分析

> **目的**：補充 MTL vs Single-Task 的計算效益比較
> 
> **建立日期**：2025-12-04
> 
> **來源**：Meeting 17 教授建議

## 1. 載入套件與資料

In [ ]:
import numpy as np
import pandas as pd
import time
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

# 假設資料已經準備好
# 這裡需要從 03_ModelBuilding.ipynb 載入資料
# X_train_scaled, X_test_scaled, y_train_multi, y_test_multi

## 2. Single-Task vs MTL 訓練時間比較

### 2.1 Logistic Regression

In [ ]:
print("="*60)
print("Logistic Regression: Single-Task vs MTL")
print("="*60)

# Single-Task
start = time.time()
lr_models = {}
for i, disease in enumerate(['高血壓', '高血糖', '高血脂']):
    model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
    model.fit(X_train_scaled, y_train_multi[:, i])
    lr_models[disease] = model
lr_single_time = time.time() - start

# MTL
start = time.time()
lr_mtl = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)
lr_mtl.fit(X_train_scaled, y_train_multi)
lr_mtl_time = time.time() - start

print(f"\nSingle-Task 訓練時間: {lr_single_time:.3f} 秒")
print(f"MTL 訓練時間: {lr_mtl_time:.3f} 秒")
print(f"時間差異: {lr_single_time - lr_mtl_time:.3f} 秒")
print(f"MTL 相對速度: {(lr_single_time / lr_mtl_time):.2f}x")

### 2.2 Random Forest

In [ ]:
print("\n" + "="*60)
print("Random Forest: Single-Task vs MTL")
print("="*60)

# Single-Task
start = time.time()
rf_models = {}
for i, disease in enumerate(['高血壓', '高血糖', '高血脂']):
    model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
    model.fit(X_train_scaled, y_train_multi[:, i])
    rf_models[disease] = model
rf_single_time = time.time() - start

# MTL (RF 原生支援多輸出)
start = time.time()
rf_mtl = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
rf_mtl.fit(X_train_scaled, y_train_multi)
rf_mtl_time = time.time() - start

print(f"\nSingle-Task 訓練時間: {rf_single_time:.3f} 秒")
print(f"MTL 訓練時間: {rf_mtl_time:.3f} 秒")
print(f"時間差異: {rf_single_time - rf_mtl_time:.3f} 秒")
print(f"MTL 相對速度: {(rf_single_time / rf_mtl_time):.2f}x")

## 3. 推論時間比較

In [ ]:
print("\n" + "="*60)
print("推論時間比較")
print("="*60)

# LR Single-Task 推論
start = time.time()
for disease, model in lr_models.items():
    _ = model.predict_proba(X_test_scaled)
lr_single_predict_time = time.time() - start

# LR MTL 推論
start = time.time()
_ = lr_mtl.predict_proba(X_test_scaled)
lr_mtl_predict_time = time.time() - start

# RF Single-Task 推論
start = time.time()
for disease, model in rf_models.items():
    _ = model.predict_proba(X_test_scaled)
rf_single_predict_time = time.time() - start

# RF MTL 推論
start = time.time()
_ = rf_mtl.predict_proba(X_test_scaled)
rf_mtl_predict_time = time.time() - start

print(f"\nLR Single-Task 推論: {lr_single_predict_time:.4f} 秒")
print(f"LR MTL 推論: {lr_mtl_predict_time:.4f} 秒")
print(f"\nRF Single-Task 推論: {rf_single_predict_time:.4f} 秒")
print(f"RF MTL 推論: {rf_mtl_predict_time:.4f} 秒")

## 4. 模型大小比較

In [ ]:
import pickle

print("\n" + "="*60)
print("模型大小比較（估計）")
print("="*60)

# LR Single-Task 大小
lr_single_size = sum(sys.getsizeof(pickle.dumps(m)) for m in lr_models.values())
lr_mtl_size = sys.getsizeof(pickle.dumps(lr_mtl))

# RF Single-Task 大小
rf_single_size = sum(sys.getsizeof(pickle.dumps(m)) for m in rf_models.values())
rf_mtl_size = sys.getsizeof(pickle.dumps(rf_mtl))

print(f"\nLR Single-Task: {lr_single_size / 1024:.2f} KB")
print(f"LR MTL: {lr_mtl_size / 1024:.2f} KB")
print(f"節省: {(lr_single_size - lr_mtl_size) / 1024:.2f} KB ({(1 - lr_mtl_size/lr_single_size)*100:.1f}%)")

print(f"\nRF Single-Task: {rf_single_size / 1024:.2f} KB")
print(f"RF MTL: {rf_mtl_size / 1024:.2f} KB")
print(f"節省: {(rf_single_size - rf_mtl_size) / 1024:.2f} KB ({(1 - rf_mtl_size/rf_single_size)*100:.1f}%)")

## 5. 綜合比較表格

In [ ]:
results = pd.DataFrame([
    {
        '模型': 'LR',
        '方法': 'Single-Task',
        '訓練時間 (秒)': lr_single_time,
        '推論時間 (秒)': lr_single_predict_time,
        '模型大小 (KB)': lr_single_size / 1024,
        '模型數量': 3
    },
    {
        '模型': 'LR',
        '方法': 'MTL',
        '訓練時間 (秒)': lr_mtl_time,
        '推論時間 (秒)': lr_mtl_predict_time,
        '模型大小 (KB)': lr_mtl_size / 1024,
        '模型數量': 1
    },
    {
        '模型': 'RF',
        '方法': 'Single-Task',
        '訓練時間 (秒)': rf_single_time,
        '推論時間 (秒)': rf_single_predict_time,
        '模型大小 (KB)': rf_single_size / 1024,
        '模型數量': 3
    },
    {
        '模型': 'RF',
        '方法': 'MTL',
        '訓練時間 (秒)': rf_mtl_time,
        '推論時間 (秒)': rf_mtl_predict_time,
        '模型大小 (KB)': rf_mtl_size / 1024,
        '模型數量': 1
    }
])

print("\n" + "="*80)
print("MTL vs Single-Task 計算效益比較")
print("="*80)
print(results.to_string(index=False))

# 儲存結果
results.to_csv('../../results/mtl_computational_efficiency.csv', index=False)
print("\n✅ 結果已儲存至 results/mtl_computational_efficiency.csv")

## 6. 結論

### 觀察

1. **訓練時間**：
   - MTL 通常比 Single-Task 快（或接近）
   - 因為只需訓練一次，而非三次

2. **推論時間**：
   - MTL 一次推論得到三個結果
   - Single-Task 需要三次獨立推論

3. **模型大小**：
   - MTL 只有一個模型
   - Single-Task 有三個獨立模型

4. **部署複雜度**：
   - MTL 更簡單（一個模型檔案）
   - Single-Task 需管理三個模型

### 但是...

- **資料量小**（~6,000 人）：效率優勢不明顯
- **效能提升有限**：見 03_ModelBuilding.ipynb 結果
- **ANN MTL 失敗**：多任務共享層導致干擾

### 論文價值

雖然 MTL 在我們的資料集上優勢不大，但：
1. 提供了完整的比較實驗
2. 證明在小資料集上 Single-Task 可能更好
3. 為未來大規模資料提供參考